In [ ]:
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
tqdm.pandas()
bho_cands = pd.read_pickle("/home/mcollardanuy/PlaceLinking/toponym_matching/ranker_results/bho_queries_britwikidata_candidates_wikigaz_en_002.pkl")
#bho_cands = bho_cands.head(1000)

britwikidata_gazetteer = pd.read_pickle("/home/mcollardanuy/PlaceLinking/toponym_matching/gazetteers/britwikidata_gazetteer.pkl")
britwikidata_gazetteer.head()

In [ ]:

def match_cands_wikidata(row,gazetteer,ranking):
    wikidata_cands = {}
    cands = row[ranking]
    for cand,score in cands.items():
        wikidataIds = gazetteer[gazetteer["altname"] == cand]["wkid"]
        for _id in wikidataIds:
            if _id not in wikidata_cands:
                wikidata_cands[_id] = score
    return wikidata_cands

tqdm.pandas()    
bho_cands['wikidata_cands'] = bho_cands.progress_apply(lambda row : match_cands_wikidata(row,britwikidata_gazetteer,"faiss_distance"), axis=1) 



In [ ]:
bho_cands.head()

In [ ]:
from wikimapper import WikiMapper
from pathlib import Path
import urllib.parse

mapper = WikiMapper("/resources/wikidata2wikipedia/index_enwiki-20190420.db")

def map_wikidata2wikidump(wikidataId):
    path = "/resources/wikipedia/extractedResources/Aspects/"
    wikititles = mapper.id_to_titles(wikidataId)
    wikititles = [urllib.parse.quote(title.replace("_"," ")) for title in wikititles]
    wikidumps = [path + title+".json" for title in wikititles if Path(path + title+".json").is_file()]
    return wikidumps


single_match = 0
single_possible_match = 0
multiple_best_match = 0
multiple_possible_match = 0
no_wiki_match = 0

cutoff = 5

for index, row in bho_cands.iterrows():
    wikidata_cands = row["wikidata_cands"]
    best_match = [x for x,y in wikidata_cands.items() if y==0.0]
    if len(best_match)>0:
        if len(best_match)==1:
            single_match+=1
        else:
            wikipages = [[wikidataId,page] for wikidataId in best_match for page in map_wikidata2wikidump(wikidataId)]
            if len(wikipages)==1:
                single_match+=1
            elif len(wikipages)>1:
                multiple_best_match+=1
            elif len(wikipages)==0:
                no_wiki_match+=1
                print (row["query"])
                print (best_match)
                print (" ")
    else:
        selected = [x for x in wikidata_cands.keys()][:cutoff]
        wikipages = [[wikidataId,page] for wikidataId in selected for page in map_wikidata2wikidump(wikidataId)]
        if len(wikipages)==1:
            single_possible_match+=1
        elif len(wikipages)>1:
            multiple_possible_match+=1
        elif len(wikipages)==0:
            no_wiki_match+=1



print ("perfect unique match:",single_match/len(bho_cands))
print ("multiple best candidates:",multiple_best_match/len(bho_cands))
print ("single possible candidate:",single_possible_match/len(bho_cands))
print ("multiple possible candidates:",multiple_possible_match/len(bho_cands))
#print ("no single best candidate:",(len(bho_cands)-single_match-multiple_match-no_wiki_match)/len(bho_cands))
print ("no wiki page available",no_wiki_match/len(bho_cands))

